# Age Classification Using Facial Images

#### Import Statements

In [1]:
import numpy as np 
import plotly.express as px 
import pandas as pd 
import cv2 
import os 
from glob import glob
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense, Dropout, LayerNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
import pickle 

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
#added import for sgd
from tensorflow.keras.optimizers import SGD

# import dlib
import dlib

import dlib

from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb

import imutils

from tensorflow.keras.metrics import Precision, Recall

e:\virtualenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Retrival and Cleaning

#### Converting the txt data to a dataframe

In [2]:
df = pd.read_csv('cleaned_IMDB_WIKI_non_gray.csv')
df

,age,gender,path,AgeRange
0,30,male,imdb_crop/99/nm0625099_rm316640512_1974-11-2_2...,Youth
1,27,male,imdb_crop/90/nm0004790_rm365789952_1976-8-23_2...,Youth
2,23,male,imdb_crop/68/nm0356468_rm3801258496_1975-8-4_1...,Youth
3,29,male,imdb_crop/58/nm1864458_rm1663605504_1975-9-17_...,Youth
4,24,male,wiki_crop/56/29835356_1989-07-25_2014.jpg,Youth
...,...,...,...,...
211096,48,male,imdb_crop/81/nm0000381_rm1343402752_1962-7-19_...,Adult
211097,42,male,imdb_crop/62/nm1101562_rm723357184_1959-10-7_2...,Adult
211098,29,female,wiki_crop/22/3571322_1977-01-31_2007.jpg,Youth
211099,57,male,imdb_crop/16/nm0000616_rm2154482176_1956-4-18_...,Old


#### Dropping null values and resampling

In [3]:
df = df.dropna()
df = df.sample(frac=1)

In [4]:
df.shape

(211101, 4)

#### Using the details of dataframe to get the image path

In [5]:
df['image_path'] = '/age and gender prediction/project/data/'+ df['path']

In [6]:
df['AgeRange'].value_counts()

Youth    104604
Adult     83935
Old       18921
Kid        3641
Name: AgeRange, dtype: int64

In [7]:
df_youth = df[df['AgeRange'] == 'Youth']
df_youth = df_youth.sample(frac = 1)
#df_youth_test1 = df_youth.tail(200)
df_youth = df_youth.head(2000)
df_youth_test = df_youth.head(500)
#df_youth_test = pd.concat([df_youth_test, df_youth_test1])

df_adult = df[df['AgeRange'] == 'Adult']
df_adult = df_adult.sample(frac = 1)
#df_adult_test1 = df_adult.tail(200)
df_adult = df_adult.head(2000)
df_adult_test = df_adult.head(500)
#df_adult_test = pd.concat([df_adult_test, df_adult_test1])

df_kid = df[df['AgeRange'] == 'Kid']
df_kid = df_kid.sample(frac = 1)
#df_kid_test1 = df_kid.tail(200)
df_kid = df_kid.head(2000)
df_kid_test = df_kid.head(500)
#df_kid_test = pd.concat([df_kid_test, df_kid_test1])

df_old = df[df['AgeRange'] == 'Old']
df_old = df_old.sample(frac = 1)
#df_old_test1 = df_old.tail(200)
df_old = df_old.head(2000)
df_old_test = df_old.head(500)
#df_old_test = pd.concat([df_old_test, df_old_test1])

df_train = pd.concat([df_youth, df_adult, df_kid, df_old])
df_test = pd.concat([df_youth_test, df_adult_test, df_kid_test, df_old_test])


In [8]:
# new_df = df[['AgeRange', 'age', 'gender', 'image_path']]

In [9]:
# del df

#### Using LabelEncoder to obtain targets in integers

In [10]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
df_train['AgeRange'] = le.fit_transform(df_train['AgeRange'])
df_test['AgeRange'] = le.fit_transform(df_test['AgeRange'])

In [11]:
df_train

,age,gender,path,AgeRange,image_path
101156,34,male,imdb_crop/35/nm0004735_rm3460017920_1977-3-8_2...,3,/age and gender prediction/project/data/imdb_c...
69181,22,female,imdb_crop/77/nm0144377_rm403806720_1978-5-11_2...,3,/age and gender prediction/project/data/imdb_c...
185985,27,female,imdb_crop/13/nm1897713_rm3462772992_1984-3-25_...,3,/age and gender prediction/project/data/imdb_c...
117380,23,female,imdb_crop/89/nm2645189_rm1986768896_1987-1-2_2...,3,/age and gender prediction/project/data/imdb_c...
121767,34,female,imdb_crop/24/nm0688624_rm768578560_1973-11-15_...,3,/age and gender prediction/project/data/imdb_c...
...,...,...,...,...,...
75167,63,female,wiki_crop/95/154695_1944-03-26_2008.jpg,2,/age and gender prediction/project/data/wiki_c...
197634,75,male,imdb_crop/28/nm0780528_rm3727862528_1913-10-28...,2,/age and gender prediction/project/data/imdb_c...
159187,61,male,imdb_crop/39/nm0122439_rm570461952_1948-1-14_2...,2,/age and gender prediction/project/data/imdb_c...
207918,61,female,imdb_crop/47/nm0001347_rm2755954432_1948-2-5_2...,2,/age and gender prediction/project/data/imdb_c...


#### Dumping the same for future usage

In [12]:
with open('imdbwiki_age_encoder_8000.pkl','wb') as pkl_file:
    pickle.dump(le, pkl_file)

## Image Preprocessing

#### Preparing to split for train and test set

In [13]:
X_train = df_train[['image_path']].values 
y_train = df_train[['AgeRange']].values 

X_test = df_test[['image_path']].values 
y_test = df_test[['AgeRange']].values 


In [14]:
set(y_train.flatten().tolist())

{0, 1, 2, 3}

In [15]:
#

#### Train Test Split

In [16]:
# from sklearn.model_selection import train_test_split 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Assigning uniform image extensions and resizing

In [17]:
def preprocess_image(individual_path):
    img = tf.io.read_file(np.array(individual_path).ravel()[0]) 
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [227,227])
    return img 

## Face Detection and Landmark Detection

In [18]:
def yield_training_values(X_train,y_train):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    fa = FaceAligner(predictor, desiredFaceWidth=256)
    for image_path, value in zip(X_train, y_train):
        imageP = image_path[0].decode("utf-8")
        img= cv2.imread(imageP, 1)
        denoised_image = cv2.fastNlMeansDenoisingColored(img, None, 5, 6, 7, 21)

        gray = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
        # Detect the face
        rects = detector(gray, 1)
        # Detect landmarks for each face
        
        try:
            for rect in rects:
                faceAligned = fa.align(img, gray, rect)

            gray1 = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(gray1, 1.3, 5)

            try:
                for (x,y,w,h) in faces:
                    # for putting rectangle on face
                    #cv2.rectangle(faceAligned, (x,y), (x+w, y+h), (0, 255, 0),3)
                    roi_color = faceAligned[y:y+h, x:x+w]
                    cv2.imwrite(imageP , roi_color)
            except:
                continue
        except:
            continue
        
        image = preprocess_image([bytes(imageP, 'utf-8')])
        yield image, value 

#### Using train and test for the model

In [19]:
ds_train = tf.data.Dataset.from_generator(yield_training_values,
                                          args=[X_train, y_train],
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=([227, 227, 3], [1]))


ds_test = tf.data.Dataset.from_generator(yield_training_values,
                                          args=[X_test, y_test],
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=([227, 227, 3], [1]))

### Shuffling the data

In [20]:
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=AUTOTUNE)
ds_test = ds_test.cache().shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=AUTOTUNE)

## Model Implementation

### Data Augmentation

In [21]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2,0.2),
])

### Model Building

In [22]:
input_shape = (227, 227, 3)

# Create a frozen base model
base_model = tf.keras.applications.InceptionV3(include_top=False, pooling = None)
for layer in base_model.layers[:-5]:
    layer.trainable = False
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Create input and output layers
inputs = tf.keras.layers.Input(shape=input_shape, name="input_layer") # create input layer
x = base_model(inputs) # pass augmented 
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.Dense(256)(x)
outputs = tf.keras.layers.Dense(4, activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(lr=0.001), # use Adam optimizer with base learning rate
              metrics=["accuracy"])

87910968/87910968 [==============================] - 8s 0us/step


e:\virtualenv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [23]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 227, 227, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 global_average_pooling_laye  (None, 2048)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 output_layer (Dense)        (None, 4)                 1028      
                                                             

In [24]:
# head = keras.models.Sequential([
#     keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=((2,2))),
#     keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=((2,2))),
#     keras.layers.Flatten(),
#     keras.layers.Dense(512, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.01)),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(4, activation='softmax')
# ])

In [25]:
# model = keras.models.Sequential([
#     data_augmentation,
#     base_model,
#     keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=((2,2))),
#     keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=((2,2))),
#     keras.layers.Flatten(),
#     keras.layers.Dense(512, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.01)),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(4, activation='softmax')
# ])

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 227, 227, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 global_average_pooling_laye  (None, 2048)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 output_layer (Dense)        (None, 4)                 1028      
                                                             

In [27]:
checkpoint_path = "model_checkpoints_weights/imdbwiki/age_checkpoint_14mar.ckpt"

# Create a ModelCheckpoint callback that saves the model's weights only
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=False, # set to False to save the entire model
                                                         save_best_only=True, # set to True to save only the best model instead of a model every epoch 
                                                         save_freq="epoch", # save every epoch
                                                         verbose=1)

In [28]:
# adam = tf.keras.optimizers.Adam(learning_rate=0.001) 
# #sgd = SGD(learning_rate=0.001)
# model.compile(optimizer=adam, loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

### Training the Model

In [ ]:
history = model.fit(ds_train, validation_data=ds_test, epochs=10, callbacks = [checkpoint_callback])

Epoch 1/10
    250/Unknown - 5353s 19s/step - loss: 49.4244 - accuracy: 0.6784
Epoch 1: val_loss improved from inf to 11.43447, saving model to model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt


INFO:tensorflow:Assets written to: model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt\assets


INFO:tensorflow:Assets written to: model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt\assets


250/250 [==============================] - 6266s 22s/step - loss: 49.4244 - accuracy: 0.6784 - val_loss: 11.4345 - val_accuracy: 0.3390
Epoch 2/10
250/250 [==============================] - ETA: 0s - loss: 3.5253 - accuracy: 0.7045
Epoch 2: val_loss improved from 11.43447 to 6.95259, saving model to model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt


INFO:tensorflow:Assets written to: model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt\assets


...

INFO:tensorflow:Assets written to: model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt\assets


250/250 [==============================] - 1836s 7s/step - loss: 1.8670 - accuracy: 0.7035 - val_loss: 3.0892 - val_accuracy: 0.2745
Epoch 5/10
250/250 [==============================] - ETA: 0s - loss: 1.2243 - accuracy: 0.7146
Epoch 5: val_loss did not improve from 3.08923
250/250 [==============================] - 1770s 7s/step - loss: 1.2243 - accuracy: 0.7146 - val_loss: 4.6985 - val_accuracy: 0.2660
Epoch 6/10
250/250 [==============================] - ETA: 0s - loss: 2.7692 - accuracy: 0.6990
Epoch 6: val_loss did not improve from 3.08923
250/250 [==============================] - 1769s 7s/step - loss: 2.7692 - accuracy: 0.6990 - val_loss: 3.3352 - val_accuracy: 0.3055
Epoch 7/10
250/250 [==============================] - ETA: 0s - loss: 1.3157 - accuracy: 0.7244
Epoch 7: val_loss did not improve from 3.08923
250/250 [==============================] - 1749s 7s/step - loss: 1.3157 - accuracy: 0.7244 - val_loss: 4.0735 - val_accuracy: 0.2555
Epoch 8/10
250/250 [===================

INFO:tensorflow:Assets written to: model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt\assets


INFO:tensorflow:Assets written to: model_checkpoints_weights/imdbwiki\age_checkpoint_14mar.ckpt\assets


250/250 [==============================] - 1773s 7s/step - loss: 1.5526 - accuracy: 0.7101 - val_loss: 2.9235 - val_accuracy: 0.3425
Epoch 9/10
242/250 [============================>.] - ETA: 53s - loss: 3.8282 - accuracy: 0.6769

In [ ]:
# Plot the validation and training data separately
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    """ 
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
model.save("models/imdbwiki/age_14mar_final.h5") 

In [ ]:
plot_loss_curves(history)

In [ ]:
model.evaluate(ds_test)

In [ ]:
model.summary()

In [ ]:
tf.keras.models.load_model(r'E:\age and gender prediction\project\models\imdbwiki\age_14mar.h5')

In [ ]:
import tensorflow as tf 
print(tf.__version__)